# Basic VertexAI pipeline

Based on this repository [here](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/pipelines_intro_kfp.ipynb)

In [1]:
!pip3 install --upgrade google-cloud-aiplatform \
                        google-cloud-storage \
                        'kfp<2' \
                        'google-cloud-pipeline-components<2' --user

In [2]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"

KFP SDK version: 1.8.22


In [3]:
PROJECT_ID = "wired-episode-388909"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [4]:
REGION = "europe-west1"  # @param {type: "string"}

In [5]:
BUCKET_NAME = f"{PROJECT_ID}-machine-learning"
BUCKET_URI = f"gs://{BUCKET_NAME}"  # @param {type:"string"}
BUCKET_URI

'gs://wired-episode-388909-machine-learning'

In [6]:
#!pip install kfp
#!pip freeze

In [7]:
from typing import NamedTuple

import google.cloud.aiplatform as aip
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import component

In [20]:
PIPELINE_ROOT = "{}/pipeline_root/intro".format(BUCKET_URI)

In [10]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

In [11]:
@component(output_component_file="hw.yaml", base_image="python:3.9")
def hello_world(text: str) -> str:
    print(text)
    return text

In [12]:
@dsl.pipeline(
    name="intro-pipeline-unique",
    description="A simple intro pipeline",
    pipeline_root=PIPELINE_ROOT,
)
def pipeline(text: str = "hi there"):
    hw_task = hello_world(text)

In [13]:
from kfp.v2 import compiler  # noqa: F811

compiler.Compiler().compile(pipeline_func=pipeline, package_path="intro_pipeline.json")

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1293: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


In [14]:
DISPLAY_NAME = "intro_pipeline_job_unique"

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="intro_pipeline.json",
    pipeline_root=PIPELINE_ROOT,
    location=REGION
)

job.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/783165767166/locations/europe-west1/pipelineJobs/intro-pipeline-unique-20230607091432
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/783165767166/locations/europe-west1/pipelineJobs/intro-pipeline-unique-20230607091432')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/europe-west1/pipelines/runs/intro-pipeline-unique-20230607091432?project=783165767166
PipelineJob run completed. Resource name: projects/783165767166/locations/europe-west1/pipelineJobs/intro-pipeline-unique-20230607091432
